<a href="https://colab.research.google.com/github/Shawn-Varghese/big-data-challenge/blob/main/DVD_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-29 21:02:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-05-29 21:02:38 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
#Putting s3 bucket and the data together
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...|2015-01-31 00:08:00|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed| 

In [5]:
#number of games
df.count()

5069140

In [6]:
#removed unnecessary points
df = df.dropna()
print(df.count())

5068421


In [ ]:
#review table
reviewdf = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
reviewdf.show(15)

In [ ]:
reviewdf.printSchema()

In [9]:
#product table 
productdf = df.select(["product_id","product_title"])
productdf = productdf.dropDuplicates()
productdf.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001DE29SS|Chaplin (15th Ann...|
|B00ZJ2GL6G|Jurassic World 3D...|
|B00T5DM876|Pretty Little Lia...|
|B004VW4V76|         Lebanon, PA|
|B009D4SFEC|Murdoch Mysteries...|
|B000GG4Y5K|Numb3rs - The Com...|
|B00383XZP8|TCM Spotlight: Ch...|
|B00I9MS86O|Breaking Bad: The...|
|B00005K3O4|Killer Klowns Fro...|
|B00BD6KETC|     Psych: Season 7|
|B00WHZZBTQ|Heaven Adores You...|
|B000092Q5C|Remo Williams - T...|
|B004AXPWH4|10 MINUTESOLUTION...|
|B00FEP9PQG|Boardwalk Empire:...|
|B0002F6BJW|Fractured Flicker...|
|B0007LXP7W|The A-Team - Seas...|
|B005ET4NHI|  Where The Boys Are|
|B005EBOSO4|Muscle Energy Tec...|
|B003V5CTKU|Black Label Socie...|
|B000087F6L|      Fahrenheit 451|
+----------+--------------------+
only showing top 20 rows



In [10]:
#grouping by customers and finding count of individual customers 
from pyspark.sql.functions import desc

customersdf = df.groupby("customer_id").agg({"customer_id":"count"})
customersdf = customersdf.orderBy(desc("count(customer_id)"))
customersdf = customersdf.withColumnRenamed("count(customer_id)", "customer_count") 
customersdf.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43430756|          3582|
|   18116317|          2985|
|   52287429|          2746|
|   52496677|          2649|
|   51110953|          2624|
|   19792742|          2495|
|   20018062|          2492|
|   50068216|          2379|
|   14539589|          2269|
|   50881246|          2104|
|   12797924|          1753|
|   50913245|          1749|
|   49382242|          1729|
|   23133606|          1729|
|   51105456|          1695|
|   38002140|          1689|
|   45838740|          1618|
|   21779173|          1501|
|   49837360|          1499|
|   17590934|          1462|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customersdf.printSchema()

In [12]:
#vine table 
from pyspark.sql.functions import col

vinedf = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])


#vinedf = vinedf.filter(col("vine")  == "Y")
vinedf.show(50)


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R33UPQQUZQEM8|          5|            3|          3|   N|
|R3IKTNQQPD9662|          5|            0|          0|   N|
|R3U27V5QMCP27T|          5|            1|          1|   N|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|
|R3OM9S0TCBP38K|          5|            0|          0|   N|
|R1W4S949ZRCTBW|          5|            0|          0|   N|
|R18JL1NNQAZFV2|          5|            0|          0|   N|
|R1LP6PR06OPYUX|          4|            0|          0|   N|
| RZKBT035JA0UQ|          5|            1|          2|   N|
|R253N5W74SM7N3|          4|            1|          1|   N|
|R2D5IFTFPHD3RN|          4|            1|          1|   N|
|R1CECK3H1URK1G|          5|            

In [13]:
vinedf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [14]:
#connecting to AWS
mode="append"
jdbc_url = "jdbc:postgresql://xxxx/dvd_db"
config = {"user": "xxxx",
          "password": "xxxx",
          "driver":"org.postgresql.Driver"}


In [15]:
reviewdf.write.jdbc(url= jdbc_url, table="review_id_table", mode=mode, properties=config)

In [16]:
productdf.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [17]:
customersdf.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
vinedf.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)